In [2]:
import requests_async as ra
import pandas as pd
import geopandas as gpd

DEFAULT_CRS = 4326
URBAN_API = 'http://10.32.1.107:5300'

In [6]:
await get_normative_service_types(1)

{46: ServiceType(id=46, accessibility_value=30.0, supply_value=150.0, accessibility_type=<AccessibilityType.MINUTES: 'мин'>, supply_type=<SupplyType.CAPACITY_PER_1000: 'ед. на 1000 человек'>, category=<Category.COMFORT: 'Комфорт'>, weight=0.2),
 49: ServiceType(id=49, accessibility_value=7.0, supply_value=61.0, accessibility_type=<AccessibilityType.MINUTES: 'мин'>, supply_type=<SupplyType.CAPACITY_PER_1000: 'ед. на 1000 человек'>, category=<Category.COMFORT: 'Комфорт'>, weight=0.2),
 50: ServiceType(id=50, accessibility_value=15.0, supply_value=120.0, accessibility_type=<AccessibilityType.MINUTES: 'мин'>, supply_type=<SupplyType.CAPACITY_PER_1000: 'ед. на 1000 человек'>, category=<Category.COMFORT: 'Комфорт'>, weight=0.2),
 47: ServiceType(id=47, accessibility_value=60.0, supply_value=13.0, accessibility_type=<AccessibilityType.MINUTES: 'мин'>, supply_type=<SupplyType.CAPACITY_PER_1000: 'ед. на 1000 человек'>, category=<Category.COMFORT: 'Комфорт'>, weight=0.2),
 48: ServiceType(id=48,

In [3]:
import shapely
import json
import requests_async as ra
import pandas as pd
import geopandas as gpd
from townsnet.provision.service_type import ServiceType

DEFAULT_CRS = 4326
URBAN_API = 'http://10.32.1.107:5300'
PAGE_SIZE = 10_000

async def _get_physical_objects(region_id : int, pot_id : int, page : int, page_size : int = PAGE_SIZE):
    res = await ra.get(f'{URBAN_API}/api/v1/territory/{region_id}/physical_objects_with_geometry', {
        'physical_object_type_id': pot_id,
        'page': page,
        'page_size': page_size,
    })
    return res.json()

async def get_physical_objects(region_id : int, pot_id : int) -> gpd.GeoDataFrame | None:
    page = 1
    results = []
    while True:
        res_json = await _get_physical_objects(region_id, pot_id, page, page_size=PAGE_SIZE)
        results.extend(res_json['results'])
        if res_json['next'] is None:
            break
        page += 1
    #recovering geometries
    for result in results:
        g = result['geometry']
        result['geometry'] = shapely.from_geojson(json.dumps(g))
    if len(results) > 0:
        return gpd.GeoDataFrame(results, crs=DEFAULT_CRS).set_index('physical_object_id')
    return None

async def get_territories(parent_id : int | None = None, all_levels = False, geometry : bool = False) -> pd.DataFrame | gpd.GeoDataFrame:
    res = await ra.get(URBAN_API + f'/api/v1/all_territories{"" if geometry else "_without_geometry"}', {
        'parent_id': parent_id,
        'get_all_levels': all_levels
    })
    res_json = res.json()
    if geometry:
        gdf = gpd.GeoDataFrame.from_features(res_json, crs=DEFAULT_CRS)
        return gdf.set_index('territory_id', drop=True)
    df = pd.DataFrame(res_json)
    return df.set_index('territory_id', drop=True)

async def get_regions(geometry : bool = False) -> gpd.GeoDataFrame:
    countries = await get_territories()
    countries_ids = countries.index
    countries_regions = [await get_territories(country_id, geometry=geometry) for country_id in countries_ids]
    return pd.concat(countries_regions) 

async def get_service_types(territory_id : int) -> list[dict]:
    res = await ra.get(URBAN_API + f'/api/v1/territory/{territory_id}/service_types')
    return res.json()

async def get_normatives(territory_id : int) -> list[dict]:
    res = await ra.get(URBAN_API + f'/api/v1/territory/{territory_id}/normatives')
    return res.json()

async def get_normative_service_types(territory_id : int) -> dict[int, ServiceType]:
    #prepare service types
    service_types = pd.DataFrame(await get_service_types(territory_id)).set_index('service_type_id')
    service_types['weight'] = service_types['properties'].apply(lambda p : p['weight_value'] if 'weight_value' in p else 0)
    service_types['category'] = service_types['infrastructure_type'].apply(str.upper)
    #prepare normatives
    normatives = pd.DataFrame(await get_normatives(territory_id))
    normatives['service_type_id'] = normatives['service_type'].apply(lambda st : st['id'])
    #merge one another
    service_types_instances = ServiceType.initialize_service_types(service_types, normatives)
    return {sti.id : sti for sti in service_types_instances}

async def get_physical_objects_types() -> list[dict]:
    res = await ra.get(URBAN_API + '/api/v1/physical_object_types')
    return res.json()

async def get_indicators():
    res = await ra.get(URBAN_API + '/api/v1/indicators_by_parent', {'get_all_subtree':True})
    return res.json()

In [4]:
from townsnet.engineering.engineering_model import EngineeringModel, EngineeringObject
import functools

ENG_OBJ_POTS = {
    EngineeringObject.ENGINEERING_OBJECT: [],
    EngineeringObject.POWER_PLANTS: [21, 33, 34, 35],
    EngineeringObject.WATER_INTAKE: [38, 40],
    EngineeringObject.WATER_TREATMENT: [37, 39],
    EngineeringObject.WATER_RESERVOIR: [],
    EngineeringObject.GAS_DISTRIBUTION: [],
}

ENG_OBJ_INDICATOR = {
  EngineeringObject.ENGINEERING_OBJECT : 88,
  EngineeringObject.POWER_PLANTS : 89,
  EngineeringObject.WATER_INTAKE : 90,
  EngineeringObject.WATER_TREATMENT : 91,
  EngineeringObject.WATER_RESERVOIR : 92,
  EngineeringObject.GAS_DISTRIBUTION : 93
}

async def _prepare_model(region_id : int) -> EngineeringModel:
    eng_objs_queries = {}
    #sending queries
    for eng_obj, pots_ids in ENG_OBJ_POTS.items():
        eng_obj_queries = []
        for pot_id in pots_ids:
            pot_query = get_physical_objects(region_id, pot_id)
            eng_obj_queries.append(pot_query)
        if len(eng_obj_queries) > 0:
            eng_objs_queries[eng_obj] = eng_obj_queries
    #awaiting queries'
    gdfs = {}
    for eng_obj, queries in eng_objs_queries.items():
        if len(queries)>0:
            queries_gdfs = [await query for query in queries]
            gdf = pd.concat(queries_gdfs)
            gdfs[eng_obj] = gdf

    return EngineeringModel(gdfs)

async def _prepare_units(region_id : int, level : int) -> gpd.GeoDataFrame:
    territories_gdf = await get_territories(region_id, all_levels = True, geometry=True)
    return territories_gdf[territories_gdf['level'] == level]

In [12]:
async def _prepare_towns(region_id : int) -> gpd.GeoDataFrame:
    territories_gdf = await get_territories(region_id, all_levels = True, geometry=True)
    levels = territories_gdf['level'].unique()
    max_level = max(levels)
    return territories_gdf[territories_gdf['level'] == max_level]

In [21]:
URBAN_API = 'http://10.32.1.107:5300'
POPULATION_COUNT_INDICATOR_ID = 1

async def get_territories_population(territories_gdf : gpd.GeoDataFrame, regional_scenario_id : int | None = None):
  res = await ra.get(f'{URBAN_API}/api/v1/indicator/{POPULATION_COUNT_INDICATOR_ID}/values')
  res_df = pd.DataFrame(res.json())
  res_df = res_df[res_df['territory_id'].isin(territories_gdf.index)]
  res_df = res_df.groupby('territory_id').agg({'value': 'last'}).rename(columns={'value':'population'})
  return territories_gdf[['geometry']].merge(res_df, left_index=True, right_index=True)

In [15]:
towns = await _prepare_towns(1)

In [20]:
await get_territories_population(towns)

/usr/lib/python3.10/json/decoder.py:353: RuntimeWarning: coroutine 'get_territories' was never awaited
  obj, end = self.scan_once(s, idx)


,geometry,population
territory_id,,
207,"POLYGON ((33.7941 59.36206, 33.79334 59.35856,...",10.0
208,"POLYGON ((33.82129 59.47496, 33.82053 59.47146...",68.0
209,"POLYGON ((33.82765 59.47334, 33.82689 59.46985...",1734.0
210,"POLYGON ((33.81098 59.44228, 33.81022 59.43878...",10.0
211,"POLYGON ((33.71243 59.32801, 33.71168 59.32451...",10.0
...,...,...
3133,"POLYGON ((31.26917 59.17068, 31.26868 59.16717...",313.0
3134,"POLYGON ((31.35423 59.18747, 31.35372 59.18396...",313.0
3135,"POLYGON ((31.50972 59.29449, 31.5092 59.29098,...",313.0
